In [10]:
import numpy as np

import hdf5plugin
import h5py

from time import perf_counter

import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
f = h5py.File('/home/andres/breakthrough-listen/data/blc40_guppi_59358_40991_HIP3740_0031.rawspec.0000.h5')
data : h5py.Dataset = f['data'] #type: ignore

In [12]:
block_indices = np.random.choice(data.shape[2] // 1024, 1_000, replace = False)
block_indices.sort()
array_indices = []
for block_index in block_indices:
    start = block_index * 1024
    end = (block_index + 1) * 1024
    array_indices += range(start, end)

load blocks individually

In [13]:
%%timeit
bit_by_bit_stds = []
for block_index in block_indices:
    start = block_index * 1024
    end = (block_index + 1) * 1024
    block = data[:, 0, start:end]
    bit_by_bit_stds.append(np.std(block))

1.49 s ± 6.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


load all blocks

In [14]:
%%timeit
all_at_once_stds = []
blocks = data[:, 0, array_indices]
for i in range(0, blocks.shape[1], 1024):
    block = blocks[:, (i):(i+1024)]
    all_at_once_stds.append(np.std(block))

13.7 s ± 344 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
